## Problem Statement 

In [ ]:
to Predict the fake news

## Data Gathering 

In [69]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [70]:
train_df = pd.read_csv("train.csv")
train_df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [71]:
test_df = pd.read_csv("test.csv")
test_df.head()

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


In [72]:
train_df.shape

(20800, 5)

In [73]:
test_df.shape

(5200, 4)

In [74]:
train_df.isnull().sum()


id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [79]:
test_df.isnull().sum()

id        0
title     0
author    0
text      0
dtype: int64

In [80]:
train_df=train_df.fillna("")
test_df= test_df.fillna("")

In [81]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5200 entries, 0 to 5199
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      5200 non-null   int64 
 1   title   5200 non-null   object
 2   author  5200 non-null   object
 3   text    5200 non-null   object
dtypes: int64(1), object(3)
memory usage: 162.6+ KB


In [82]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20800 non-null  object
 2   author  20800 non-null  object
 3   text    20800 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [83]:
## merging the column of title and author
train_df["content"]=train_df["author"]+" "+train_df["title"]
test_df["content"]=test_df["author"]+" "+test_df["title"]

In [84]:
train_df.head()

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...


In [85]:
test_df.head()

,id,title,author,text,content
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning...",David Streitfeld Specter of Trump Loosens Tong...
1,20801,Russian warships ready to strike terrorists ne...,,Russian warships ready to strike terrorists ne...,Russian warships ready to strike terrorists n...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...,Common Dreams #NoDAPL: Native American Leaders...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different...",Daniel Victor Tim Tebow Will Attempt Another C...
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...,Truth Broadcast Network Keiser Report: Meme Wa...


In [86]:
x = train_df.drop(columns="label", axis=1)
y = train_df["label"]

In [87]:
x

,id,title,author,text,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,Howard Portnoy Iranian woman jailed for fictio...
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,"Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma..."
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,Michael J. de la Merced and Rachel Abrams Macy...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...","Alex Ansary NATO, Russia To Hold Parallel Exer..."


In [88]:
y

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 20800, dtype: int64

# stemming

In [89]:
port_stem = PorterStemmer()

In [90]:
def stemming(content):
    stemm_content = re.sub("[^a-zA-Z]", ' ', content)
    stemm_content = stemm_content.lower()
    stemm_content = stemm_content.split()
    stemm_content=[port_stem.stem(word)for word in stemm_content if not word in stopwords.words("english")]
    stemm_content = "".join(stemm_content)
    return stemm_content

In [91]:
train_df["content"]=train_df["content"].apply(stemming)

In [92]:
train_df["content"]

0        darrellucuhousdemaidevenseecomeyletterjasoncha...
1        danieljflynnflynnhillariclintonbigwomancampubr...
2                        consortiumnewcomtruthmightgetfire
3        jessicapurkissciviliankillsinglusairstrikidentifi
4        howardportnoyiranianwomanjailfictionunpublishs...
                               ...                        
20795      jeromhudsonrappertrumpposterchildwhitesupremaci
20796    benjaminhoffmannflplayoffschedulmatchupoddnewy...
20797    michaeljdelamercrachelabrammacisaidreceivtakeo...
20798        alexansarinatorussiaholdparallelexercisbalkan
20799                                davidswansonkeepfaliv
Name: content, Length: 20800, dtype: object

In [93]:
x = train_df["content"].values
y = train_df["label"].values

In [94]:
x

array(['darrellucuhousdemaidevenseecomeyletterjasonchaffetztweet',
       'danieljflynnflynnhillariclintonbigwomancampubreitbart',
       'consortiumnewcomtruthmightgetfire', ...,
       'michaeljdelamercrachelabrammacisaidreceivtakeovapproachhudsonbaynewyorktime',
       'alexansarinatorussiaholdparallelexercisbalkan',
       'davidswansonkeepfaliv'], dtype=object)

In [95]:
y

array([1, 0, 1, ..., 0, 1, 1], dtype=int64)

In [96]:
vectorizer = TfidfVectorizer()
vectorizer.fit(x)
x=vectorizer.transform(x)

In [97]:
print(x)

  (0, 5358)	1.0
  (1, 5051)	1.0
  (2, 4667)	1.0
  (3, 10068)	1.0
  (4, 8336)	1.0
  (5, 5141)	1.0
  (6, 12328)	1.0
  (7, 1003)	1.0
  (8, 7111)	1.0
  (9, 13247)	1.0
  (10, 85)	1.0
  (11, 4358)	1.0
  (12, 1157)	1.0
  (13, 9431)	1.0
  (14, 1691)	1.0
  (15, 9059)	1.0
  (16, 13377)	1.0
  (17, 17920)	1.0
  (18, 5952)	1.0
  (19, 9596)	1.0
  (20, 14535)	1.0
  (21, 9979)	1.0
  (22, 15437)	1.0
  (23, 13050)	1.0
  (24, 6030)	1.0
  :	:
  (20775, 1820)	1.0
  (20776, 9289)	1.0
  (20777, 412)	1.0
  (20778, 14343)	1.0
  (20779, 18568)	1.0
  (20780, 4234)	1.0
  (20781, 8106)	1.0
  (20782, 5955)	1.0
  (20783, 4049)	1.0
  (20784, 5703)	1.0
  (20785, 1607)	1.0
  (20786, 7814)	1.0
  (20787, 8589)	1.0
  (20788, 10187)	1.0
  (20789, 19697)	1.0
  (20790, 291)	1.0
  (20791, 5016)	1.0
  (20792, 10566)	1.0
  (20793, 16782)	1.0
  (20794, 12191)	1.0
  (20795, 9993)	1.0
  (20796, 2412)	1.0
  (20797, 13456)	1.0
  (20798, 885)	1.0
  (20799, 5619)	1.0


In [98]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.25, stratify=y, random_state=11)

In [99]:
model = LogisticRegression()

In [100]:
model.fit(x_train,y_train)

LogisticRegression()

In [101]:
y_pred_train = model.predict(x_train)
acc = accuracy_score(y_pred_train, y_train)

In [102]:
print("Accuracy Score ", acc)

Accuracy Score  0.9934615384615385


In [103]:
y_pred_test = model.predict(x_test)
acc = accuracy_score(y_pred_test, y_test)

In [104]:
print("Accuracy Score ", acc)

Accuracy Score  0.5380769230769231
